In [19]:
import psycopg2 as pg
import pandas as pd
from multiprocessing import Pool

from equities_data import create_feature_df
from config import DATABASE_URI

In [20]:
# Select all tickers that are in US dollars, are common stock, and have at least five years
# of price and fundamental data
sql = """
SELECT
    ticker
FROM
    tickers
WHERE
    (
        EXTRACT(
            YEAR
            FROM
                lastpricedate
        ) - EXTRACT(
            YEAR
            FROM
                firstpricedate
        ) >= 5
        AND EXTRACT(
            YEAR
            FROM
                lastquarter
        ) - EXTRACT(
            YEAR
            FROM
                firstquarter
        ) >= 5
        AND category = 'Domestic Common Stock'
        AND currency = 'USD'
    );
"""

with pg.connect(DATABASE_URI) as conn:
    with conn.cursor() as cur:
        cur.execute(sql)
        results = cur.fetchall()

tickers = [n[0] for n in results]

In [21]:
with Pool(processes=4) as pool:
    result = pool.map(create_feature_df, tickers)

/home/eadains/EquitiesTrading/equities_data.py:178: RuntimeWarning: divide by zero encountered in double_scalars
  (df.iloc[-1]["assetsc"] - df.iloc[-1]["cash"] - df.iloc[-1]["liabilitiesc"])
/home/eadains/EquitiesTrading/equities_data.py:178: RuntimeWarning: divide by zero encountered in double_scalars
  (df.iloc[-1]["assetsc"] - df.iloc[-1]["cash"] - df.iloc[-1]["liabilitiesc"])
/home/eadains/EquitiesTrading/equities_data.py:178: RuntimeWarning: divide by zero encountered in double_scalars
  (df.iloc[-1]["assetsc"] - df.iloc[-1]["cash"] - df.iloc[-1]["liabilitiesc"])
Process ForkPoolWorker-27:
Process ForkPoolWorker-31:
Process ForkPoolWorker-30:
Process ForkPoolWorker-28:
Traceback (most recent call last):


KeyboardInterrupt: 